<a href="https://colab.research.google.com/github/Bharathibusaboina/Applied-DataScience-Capstone-Projecct/blob/main/spacex_dash_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
!pip install dash jupyter_dash 
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

In [ ]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# Read the airline data into pandas dataframe
#"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
max_payload = spacex_df['Payload Mass (kg)'].max()
print(max_payload)
min_payload = spacex_df['Payload Mass (kg)'].min()
print(min_payload)

9600.0
0.0


In [ ]:
def get_succeessdata_for_launchsite(space_data, enteredsite):
  space_data = spacex_df
  if (enteredsite == 'OPT1'):
    pie_data_launch_sites = space_data[["Launch Site", "class"]].groupby("Launch Site").mean()      
    scatter_data_launch_sites = space_data
  else:    
    pie_data_launch_sites = space_data[ ( space_data["Launch Site"] == enteredsite ) ].groupby("class").count()    
    scatter_data_launch_sites = space_data[space_data["Launch Site"] == enteredsite]
    
  return pie_data_launch_sites, scatter_data_launch_sites


In [ ]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 30}),                                
                                html.Div([                                    
                                    html.Div([                                        
                                        html.Div([html.H2('Launch Site:', style={'margin-right': '2em'}),]),                                                                                                                
                                        dcc.Dropdown(id='site-dropdown',                                                      
                                                     options=[{'label': 'ALL Sites', 'value': 'OPT1'}, {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, 
                                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}],
                                                     value='OPT1',  
                                                     placeholder="Select a Launch Site here",  
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),                                                                            
                                    ], style={'display':'flex'}),                                                                                                   
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),                                
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000,   
                                                marks={ 0: {'label': '0KG', 'style': {'color': '#77b0b1'}},
                                                        2000: {'label': '2000 KG'},                                                       
                                                        4000: {'label': '4000 KG'},
                                                        6000: {'label': '6000 KG'}, 
                                                        8000: {'label': '8000 KG'},
                                                        10000: {'label': '10000 KG'}},
                                                value=[0, 9600]),                                       
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ]),
                                # signal value to trigger callbacks
                                ])                         


In [ ]:
@app.callback( [Output(component_id='success-pie-chart', component_property='figure'),
                Output(component_id='success-payload-scatter-chart', component_property='figure')],              
               [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id='payload-slider', component_property='value')] )                
              
def get_graph(launchsite, payload):      

  pie_plot_data, scatter_plot_data = get_succeessdata_for_launchsite(spacex_df, launchsite)
  if ( launchsite == 'OPT1'):       
    pie_plot=px.pie(pie_plot_data, values="class", names=pie_plot_data.index, title = "successful flights of all Launchsites") 
    scatter_plot = px.scatter(scatter_plot_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    scatter_plot.update_layout(title='Payload vs success', xaxis_title='Payload Mass (kg)', yaxis_title='class')        
  else:       
    pie_plot=px.pie(pie_plot_data, values="Launch Site", names=pie_plot_data.index, title = "successful or Failure of flights at Launchsite")
    scatter_plot = px.scatter(scatter_plot_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
    scatter_plot.update_layout(title='Payload vs success', xaxis_title='Payload Mass (kg)', yaxis_title='class')

  return[pie_plot, scatter_plot]    

In [ ]:
if __name__ == '__main__':    
    app.run_server(host="localhost", port=7619, debug=True)


Dash app running on:


<IPython.core.display.Javascript object>